In [6]:
#zeta = 0.1
#eta = 0.5
#u_0 = 1.0
#v_0 = 0.0
#a_s = 0.1
#a_c = 0
#N_end = 3.0

In [2]:
from ipywidgets import widgets
from ipywidgets import interact

In [1]:
def interactive_tau_u_plot():
    @interact(i_zeta=widgets.IntSlider(min=0, max=250, value=50, step=5, description='$\zeta$ (1/100):'),
              i_eta = widgets.IntSlider(min=5, max=505, value=5, step=10, description='$\eta$ (1/100):'),
              i_u0=widgets.IntSlider(min=-25, max=25, value=10, step=1, description='$u_0$ (1/10):'),
              i_v0=widgets.IntSlider(min=-25, max=25, value=0, step=1, description='$v_0$:(1/10) '),
              i_as = widgets.IntSlider(min=-25, max=25, value=0, step=1, description='$a_s$ (1/10):'),
              i_ac = widgets.IntSlider(min=-25, max=25, value=0, step=1, description='$a_c$ (1/10):'),
              i_Nend = widgets.IntSlider(min=1, max=50, value=5, step=1, description='$\\tau_{end}$ (T):')
             )

    def plot_now(i_zeta, i_eta, i_u0, i_v0, i_as, i_ac, i_Nend):
        zeta = i_zeta/100.0
        eta = i_eta/100.0
        u_0 = i_u0/10.0
        v_0 = i_v0/10.0
        a_s = i_as/10.0
        a_c = i_ac/10.0
        N_end = i_Nend
        
        a = lambda tau: a_c*np.cos(eta*tau) + a_s*np.sin(eta*tau)    

        [u_c, u_s, V, psi] = steady_state_solution(eta, 1.0, zeta, a_c, a_s)
        up = lambda tau: u_c*np.cos(eta*tau) + u_s*np.sin(eta*tau)
        vp = lambda tau: (-u_c*np.sin(eta*tau) + u_s*np.cos(eta*tau)) * eta
     
        if zeta < 1:
            omega_1 = omega_0*np.sqrt(1-zeta**2)
            delta = omega_0*zeta
            uh_1 = lambda tau: np.exp(-delta*tau)*np.cos(omega_1*tau)
            uh_2 = lambda tau: np.exp(-delta*tau)*np.sin(omega_1*tau)
            vh_1 = lambda tau: np.exp(-delta*tau)*( -delta*np.cos(omega_1*tau) - omega_1*np.sin(omega_1*tau) )
            vh_2 = lambda tau: np.exp(-delta*tau)*( -delta*np.sin(omega_1*tau) + omega_1*np.cos(omega_1*tau) )
        elif zeta > 1:
            delta_1 = zeta - np.sqrt(zeta**2 - 1)
            delta_2 = zeta + np.sqrt(zeta**2 - 1)
            uh_1 = lambda tau: np.exp(-delta_1*tau)
            uh_2 = lambda tau: np.exp(-delta_2*tau)
            vh_1 = lambda tau: np.exp(-delta_1*tau)*(-delta_1)
            vh_2 = lambda tau: np.exp(-delta_2*tau)*(-delta_2)
        else:
            delta = zeta
            uh_1 = lambda tau: np.exp(-delta*tau)
            uh_2 = lambda tau: tau*np.exp(-delta*tau)
            vh_1 = lambda tau:-np.exp(-delta*tau)*delta
            vh_2 = lambda tau: np.exp(-delta*tau) - tau*np.exp(-delta*tau)*delta     
        [C_1, C_2] = transient_solution(0.0, u_0, v_0)
        uh = lambda tau: C_1*uh_1(tau) + C_2*uh_2(tau) 
        u = lambda tau:  uh(tau) + up(tau)        
        
        tau_end = N_end*2*np.pi
        tau_sample = np.linspace(0, tau_end , N_end*50)   # discretization with 250 points in time
        a_sample = a(tau_sample)
        uh_sample = uh(tau_sample)
        up_sample = up(tau_sample)
        u_sample = u(tau_sample)

        #plt.plot(t_sample, uh_sample, 'b--', t_sample, up_sample, 'b:', t_sample, u_sample, 'b');
        fig, ax1 = plt.subplots()
        plt.box(on=True)
        color = 'tab:blue'
        ax1.set_xlabel('$\\tau$')
        ax1.set_ylabel('$u$', color=color)
        ax1.plot(tau_sample, u_sample, color=color)
        ax1.tick_params(axis='y', labelcolor=color)
        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        color = 'tab:red'
        ax2.spines['right'].set_visible(True)
        ax2.spines['top'].set_visible(True)
        ax2.set_ylabel('$a$', color=color)  # we already handled the x-label with ax1
        ax2.plot(tau_sample, a_sample, color=color)
        ax2.tick_params(axis='y', labelcolor=color) 
        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        plt.show()